In [42]:
import sys
sys.dont_write_bytecode = True

from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas

import matplotlib.pyplot as plt


# Local imports.
from resale_flat_prices.resale_flat_data.resale_flat_data import ResaleFlatData
from resale_flat_prices.resale_flat_data.rent_prices_data import RentPricesData
from resale_flat_prices.vis_utils.vis_utils import plot_df


# Data directories.
csv_data_dir = Path("../data/ResaleFlatPrices/")
processed_data_dir = Path("../data/processed_data/")

# Load processed and geocoded resale flat data.
resale_flat_data_csv_file = "resale-flat-prices-indexed.parquet"
rent_prices_data_csv_file = "rent-prices-indexed.parquet"

# H3 cell creation using a specified resolution.
RESOLUTION = 9

In [ ]:
# Resale flat data (indexed).

price_col = "resale_price" #"price_per_sqft"
price_index = "price_index"
price_col_indexed = price_col + "_indexed"

resale_flat_data = ResaleFlatData(processed_data_dir / resale_flat_data_csv_file)
resale_flat_data.read_parquet()
resale_flat_data.df = resale_flat_data.df.sort_values(["datetime", "town", "street_name", "block"])
resale_flat_data.df = resale_flat_data.df.reset_index(drop = True)

#resale_flat_data.df = resale_flat_data.df[resale_flat_data.df["town"] == "TOA PAYOH"]

resale_flat_data.df[price_col_indexed] = resale_flat_data.df[price_col] / resale_flat_data.df[price_index]

resale_flat_data.make_h3_geometries(resolution = RESOLUTION)

#display(resale_flat_data.df.head())
print("Loaded resale_flat_data.df.shape: {}.".format(resale_flat_data.df.shape))

In [ ]:
# Rent data (indexed).

rent_col = "monthly_rent"
rent_index = "rent_index"
rent_col_indexed = rent_col + "_indexed"

rent_data = RentPricesData(processed_data_dir / rent_prices_data_csv_file)
rent_data.read_parquet()
rent_data.df = rent_data.df.sort_values(["datetime", "town", "street_name", "block"])
rent_data.df = rent_data.df.reset_index(drop = True)

rent_data.df[rent_col_indexed] = rent_data.df[rent_col] / rent_data.df[rent_index]

rent_data.make_h3_geometries(resolution = RESOLUTION)

#display(rent_data.df.head())
print("Loaded rent_data.df.shape: {}.".format(rent_data.df.shape))

In [62]:
def quantile(X, q = 0.5, axis = None):
    return np.quantile(X, q=q, axis=axis)


median_resale_prices_df = resale_flat_data.df[
    ["geometry", price_col_indexed]
].groupby(["geometry"]).aggregate("median").reset_index()

median_resale_prices_df = geopandas.GeoDataFrame(median_resale_prices_df)
median_resale_prices_df = median_resale_prices_df.set_crs(resale_flat_data.df.crs)

median_resale_prices_df[price_col_indexed] = median_resale_prices_df[price_col_indexed].apply(lambda x: int(round(x)))

In [63]:
median_rent_df = rent_data.df[
    ["geometry", rent_col_indexed]
].groupby(["geometry"]).aggregate("median").reset_index()

median_rent_df = geopandas.GeoDataFrame(median_rent_df)
median_rent_df = median_rent_df.set_crs(rent_data.df.crs)

median_rent_df[rent_col_indexed] = median_rent_df[rent_col_indexed].apply(lambda x: int(round(x)))

In [83]:
median_df = median_resale_prices_df.merge(
    median_rent_df, how = "outer", on = "geometry",
)

median_df["rent_price_ratio"] = median_df[rent_col_indexed] / median_df[price_col_indexed]

median_df[price_col_indexed] = median_df[price_col_indexed] / 100000

In [ ]:
plot_df(
    median_df[["geometry", price_col_indexed]], 
    plot_kwds = {
        "figsize": [12, 12],
        "xlim": [103.60152080468028, 104.0470051248534],
        "ylim": [1.2359029533199608, 1.4733321131970046],
        "title": "{}".format("Median HDB resale price [100k SGD] per sqft."),
        "alpha": 0.75,
        "categorical": False,
        "column": price_col_indexed,
        "legend": True,
        "legend_kwds": {"label": "Price [100k SGD] per sqft."},
        "cmap": 'viridis',
        "edgecolor": None,
        "divider_kwds": {"position": "right", "size": "5%", "pad": 0.1},
    },
)

In [ ]:
plot_df(
    median_df[["geometry", rent_col_indexed]], 
    plot_kwds = {
        "figsize": [12, 12],
        "xlim": [103.60152080468028, 104.0470051248534],
        "ylim": [1.2359029533199608, 1.4733321131970046],
        "title": "{}".format("Median rent [SGD]."),
        "alpha": 0.75,
        "categorical": False,
        "column": rent_col_indexed,
        "legend": True,
        "legend_kwds": {"label": "Rent [SGD]."},
        "cmap": 'viridis',
        "edgecolor": None,
        "divider_kwds": {"position": "right", "size": "5%", "pad": 0.1},
    },
)